In [2]:
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [5]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20465
Most common words [('the', 10129), ('a', 7312), ('and', 6199), ('of', 6063), ('to', 4233), ('is', 3378)]
Sample data [4, 669, 9, 2800, 8, 22, 4, 3556, 15739, 98] ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'centurys', 'new']


In [6]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [13]:
class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate,
                top_k=5, n_filters=250):
        kernels = [3, 4, 5]
        
        def add_conv1d(x, n_filters, kernel_size, strides=1):
            return tf.layers.conv1d(inputs = x,
                            filters = n_filters,
                            kernel_size  = kernel_size,
                            strides = strides,
                            padding = 'valid',
                            use_bias = True,
                            activation = tf.nn.relu)
        
        def add_kmax_pooling(x):
            Y = tf.transpose(x, [0, 2, 1])
            Y = tf.nn.top_k(Y, top_k, sorted=False).values
            Y = tf.transpose(Y, [0, 2, 1])
            return tf.reshape(Y, [-1, top_k, n_filters//len(kernels)])
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        parallels = []
        for k in kernels:
            p = add_conv1d(encoder_embedded, n_filters//len(kernels), kernel_size=k)
            p = add_kmax_pooling(p)
            parallels.append(p)
            
        parallels = tf.concat(parallels, axis=-1)
        parallels = tf.reshape(parallels, [-1, top_k * (len(kernels)*(n_filters//len(kernels)))])
        feed = tf.layers.dense(parallels, n_filters, tf.nn.relu)
        self.logits = tf.layers.dense(parallels, dimension_output)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

In [15]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.510254
time taken: 12.449951410293579
epoch: 0, training loss: 0.726053, training acc: 0.561435, valid loss: 0.737590, valid acc: 0.510254

epoch: 1, pass acc: 0.510254, current acc: 0.517578
time taken: 11.64803957939148
epoch: 1, training loss: 0.519731, training acc: 0.764915, valid loss: 0.762266, valid acc: 0.517578

epoch: 2, pass acc: 0.517578, current acc: 0.528320
time taken: 11.61558723449707
epoch: 2, training loss: 0.404359, training acc: 0.858428, valid loss: 0.826527, valid acc: 0.528320

time taken: 11.677389144897461
epoch: 3, training loss: 0.316785, training acc: 0.901752, valid loss: 0.896225, valid acc: 0.498047

time taken: 11.632940530776978
epoch: 4, training loss: 0.225914, training acc: 0.953362, valid loss: 0.941985, valid acc: 0.503906

time taken: 11.703073024749756
epoch: 5, training loss: 0.169737, training acc: 0.975142, valid loss: 1.018071, valid acc: 0.519531

time taken: 11.649460315704346
epoch: 6, trainin

In [16]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.72      0.64      0.68      1055
   positive       0.68      0.76      0.72      1078

avg / total       0.70      0.70      0.70      2133

